In [3]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from IPython.display import clear_output, display
from time import sleep as sl

In [ ]:
import http.client
http.client._MAXHEADERS = 1000

from requests_html import AsyncHTMLSession
URL = 'https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory'
asession = AsyncHTMLSession()
response = await asession.get(URL)
await response.html.arender(wait=2, timeout=100)
soup = BeautifulSoup(response.html.html, 'lxml')
print(soup.prettify())

In [ ]:
states = [x.attrs['href'][-3:-1] for x in soup.find_all('a', {'href': re.compile("https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/\w{2}/")})]
states

In [ ]:
cities = {}
for state in states:
    current_state_url = f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/{state}/"
    response = await asession.get(current_state_url)
    await response.html.arender(wait=2, timeout=100)
    current_state_soup = BeautifulSoup(response.html.html, 'lxml')

    cities[f"{state}"] = [x.attrs['href'][68:-1] for x in current_state_soup.find_all('a', {'href': re.compile(f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/az/.")})]
    print(state)

    sl(2)

print(cities)

In [ ]:
current_state_url = f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/az/"
response = await asession.get(current_state_url)
await response.html.arender(wait=2, timeout=100)
current_state_soup = BeautifulSoup(response.html.html, 'lxml')

print([x.attrs['href'][68:-1] for x in current_state_soup.find_all('a', {'href': re.compile(f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/az/.")})])

In [ ]:
current_city_url = f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/az/chandler"
response = await asession.get(current_city_url)
await response.html.arender(wait=2, timeout=100)
current_city_soup = BeautifulSoup(response.html.html, 'lxml')

i = current_city_soup.find('div', {'class': 'col-md-17'})

In [ ]:
i.div.contents
try:
    while True:
        i.div.contents.remove('\n')
except ValueError:
    pass

address = ''
for i, clinic in enumerate(i.div.contents):
    if i%2 == 0:
        address = clinic.text[:-12]
        print(address)
    else:
        services = '\n'.join([x.text for x in clinic.find_all('a')])
        print(services)
#         print([x.text for x in clinic.find_all('a')])

In [ ]:
i = current_city_soup.find('div', {'class': 'col-md-17'})
i.div.contents[1].find_all('a')

In [ ]:
import http.client
http.client._MAXHEADERS = 1000

from requests_html import AsyncHTMLSession
URL = 'https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory'
asession = AsyncHTMLSession()
response = await asession.get(URL)
await response.html.arender(wait=2, timeout=100)
soup = BeautifulSoup(response.html.html, 'lxml')

states = [x.attrs['href'][-3:-1] for x in soup.find_all('a', {'href': re.compile("https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/\w{2}/")})]
df = []

for state in states:
    current_state_url = f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/{state}/"
    response = await asession.get(current_state_url)
    await response.html.arender(wait=2, timeout=100)
    current_state_soup = BeautifulSoup(response.html.html, 'lxml')

    cities = [x.attrs['href'][68:-1] for x in current_state_soup.find_all('a', {'href': re.compile(f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/az/.")})]
    for city in cities[:6]:
        current_city_url = f"https://www.cvs.com/minuteclinic/clinic-locator/clinic-directory/{state}/{city}"
        response = await asession.get(current_city_url)
        await response.html.arender(wait=2, timeout=100)
        current_city_soup = BeautifulSoup(response.html.html, 'lxml')

        clinics_list = current_city_soup.find('div', {'class': 'col-md-17'})
#         print(clinics_list)
        
        clinics_list.div.contents
        try:
            while True:
                clinics_list.div.contents.remove('\n')
        except ValueError:
            pass

        address = ''
        for i, clinic in enumerate(clinics_list.div.contents):
            if i%2 == 0:
                address = clinic.text[:-12]
                print(address)
            else:
                services = '\n'.join([x.text for x in clinic.find_all('a')])
                df.append([address, services])
#                 print(services)
#                 print([x.text for x in clinic.find_all('a')])
        sl(1)
    sl(2)


28138 North Tatum Boulevard, Cave Creek, AZ 85331
2010 South Dobson Road, Chandler, AZ 85286
2995 East Chandler Heights Road, Chandler, AZ 85249
5975 West Chandler Boulevard, Chandler, AZ 85226
3170 South Higley Road, Gilbert, AZ 85295
765 South Lindsay Road, Gilbert, AZ 85296
18591 North 59th Avenue, Glendale, AZ 85308
2840 North Dysart Road, Goodyear, AZ 85395


RuntimeError: This event loop is already running

In [1]:
pd.DataFrame(df)

NameError: name 'pd' is not defined